In [1]:
import os
import anndata
import pandas as pd
import numpy as np
import scanpy as sc

from mgitools.os_helpers import listfiles

#### prepare inputs for bayesprism

In [5]:
sct = sc.read_h5ad('../data/single_cell/checkpoints/non_eus_processed.h5ad')
sct

AnnData object with n_obs × n_vars = 113052 × 29227
    obs: 'orig.ident', 'nCount_RNA', 'nFeature_RNA', 'percent.mt', 'nCount_SCT', 'nFeature_SCT', 'CELL', 'CONDITION', 'Patient', 'Type', 'Cell_type', 'integrated_snn_res.0.75', 'seurat_clusters', 'sample_id', 'cell_type', 'pollock_cell_type', 'dataset', 'Bailey | ADEX | score', 'Bailey | Squamous-like | score', 'Bailey | Pancreatic-Progenitor | score', 'Bailey | Immunogenic | score', 'Collison | Exocrine-like | score', 'Collison | Quasi-Mesenchymal | score', 'Collison | Classical | score', 'Moffit | Basal | score', 'Moffit | Classical | score', 'subTME | deserted | score', 'subTME | reactive | score', 'raghaven | scBasal | score', 'raghaven | scClassical | score', 'raghaven | IC | score', 'raghaven | Pericyte-like | score', 'raghaven | Fibroblast-like | score', 'raghaven | Inflammatory | score', 'raghaven | TAM-FCN1 | score', 'raghaven | TAM-C1QC | score', 'raghaven | TAM-SPP1 | score', 'elyada | myCAF | score', 'elyada | iCAF | score

In [6]:
set(sct.obs['cell_type_specific_final'])

{'ADM',
 'Acinar',
 'B cell',
 'CD4 T cell',
 'CD8 T cell',
 'CD8 T cell - Exhausted',
 'DC',
 'Endocrine',
 'Endothelial',
 'Exclude - Ambiguous',
 'Exclude - Singleton',
 'Immune - Proliferating',
 'Malignant - Basal',
 'Malignant - Classical',
 'Malignant - IC',
 'Malignant - Proliferating Basal',
 'Malignant - Proliferating Classical',
 'Malignant - Proliferating IC',
 'Mast',
 'NK',
 'Plasma',
 'Stellate',
 'TAM - C1QC',
 'TAM - FCN1',
 'TAM - Proliferating',
 'TAM - SPP1',
 'Treg',
 'iCAF',
 'myCAF'}

In [7]:
sct = sct[[True if 'Exclude' not in ct else False
          for ct in sct.obs['cell_type_specific_final']]]
sct.shape

(104486, 29227)

In [9]:
ref_adata = anndata.AnnData(X=sct.layers['counts'],
                            obs=sct.obs[['cell_type_specific_final']],
                           var=sct.var)
ref_adata

/diskmnt/Projects/Users/estorrs/miniconda3/envs/single-cell-pdac/lib/python3.7/site-packages/ipykernel_launcher.py:3: FutureWarning: X.dtype being converted to np.float32 from float64. In the next version of anndata (0.9) conversion will not be automatic. Pass dtype explicitly to avoid this warning. Pass `AnnData(X, dtype=X.dtype, ...)` to get the future behavour.
  This is separate from the ipykernel package so we can avoid doing imports until


AnnData object with n_obs × n_vars = 104486 × 29227
    obs: 'cell_type_specific_final'

In [10]:
ref_adata.obs['tumor_flag'] = [1 if 'Malignant' in ct else 0
                              for ct in ref_adata.obs['cell_type_specific_final']]

In [11]:
ref_adata.obs.columns = ['cell_type', 'tumor_flag']
ref_adata.obs.index.name = 'cell_ID'
ref_adata.obs

,cell_type,tumor_flag
cell_ID,,
1555-tumor_AAACCTGAGACCTAGG-1,NK,0
1555-tumor_AAACCTGAGTGCGTGA-1,Malignant - Classical,1
1555-tumor_AAACCTGCATCCCACT-1,Treg,0
1555-tumor_AAACCTGGTCATGCAT-1,B cell,0
1555-tumor_AAACCTGTCCGGGTGT-1,CD8 T cell,0
...,...,...
G9903_filtered_gene_bc_matrices_h5.h5_TTTGGTTTCCTAGTGA-1,iCAF,0
G9903_filtered_gene_bc_matrices_h5.h5_TTTGGTTTCTACCAGA-1,myCAF,0
G9903_filtered_gene_bc_matrices_h5.h5_TTTGTCAAGTTGTCGT-1,myCAF,0


In [12]:
# # subsample to 5k cells per cell type
cell_types = sorted(set(sct.obs['cell_type_specific_final']))
pool = []
for ct in cell_types:
    if 'Exclude' not in ct:
        f = sct[sct.obs['cell_type_specific_final']==ct]
        ids = list(np.random.choice(f.obs.index.to_list(), size=min(500, f.shape[0]), replace=False))
        pool += ids
f = ref_adata[pool]
f = f[:, np.sum(f.X, axis=0)>10]
f

View of AnnData object with n_obs × n_vars = 13028 × 21657
    obs: 'cell_type', 'tumor_flag'

In [13]:
bulk_fps = sorted(listfiles('../data/bulk_rna_seq/', regex=r'bulk_rna_seq/[^/]+_counts.txt$'))
bulk_fps

['../data/bulk_rna_seq/bailey_counts.txt',
 '../data/bulk_rna_seq/cptac_counts.txt',
 '../data/bulk_rna_seq/kirby_counts.txt',
 '../data/bulk_rna_seq/tcga_counts.txt']

In [14]:
genes = set(f.var.index.to_list())
for fp in bulk_fps:
    df = pd.read_csv(fp, sep='\t', index_col=0)
    genes.intersection_update(set(df.index.to_list()))
    print(fp, len(genes))
len(genes)

../data/bulk_rna_seq/bailey_counts.txt 16906
../data/bulk_rna_seq/cptac_counts.txt 16495
../data/bulk_rna_seq/kirby_counts.txt 14073
../data/bulk_rna_seq/tcga_counts.txt 14048


14048

In [15]:
f = f[:, sorted(genes)]
f

View of AnnData object with n_obs × n_vars = 13028 × 14048
    obs: 'cell_type', 'tumor_flag'

In [16]:
f.obs

,cell_type,tumor_flag
cell_ID,,
T9_CACATTTAGAGTAATC,ADM,0
T15_CTACCCAGTCACTGGC,ADM,0
T2_ATTCTACAGTAATCCC,ADM,0
T2_CTCGGGATCATAACCG,ADM,0
T2_GGTGCGTTCAAACCAC,ADM,0
...,...,...
87784_filtered_gene_bc_matrices_h5.h5_GGTGTTAGTTTAAGCC-1,myCAF,0
T11_CTTACCGGTATTCTCT,myCAF,0
H_ZY-1174-06_CCCATACGTTTCGCTC-1,myCAF,0


In [17]:
f.obs.index = ['X' + x for x in f.obs.index.to_list()]
f.obs.index.name = 'cell_ID'
f.obs

,cell_type,tumor_flag
cell_ID,,
XT9_CACATTTAGAGTAATC,ADM,0
XT15_CTACCCAGTCACTGGC,ADM,0
XT2_ATTCTACAGTAATCCC,ADM,0
XT2_CTCGGGATCATAACCG,ADM,0
XT2_GGTGCGTTCAAACCAC,ADM,0
...,...,...
X87784_filtered_gene_bc_matrices_h5.h5_GGTGTTAGTTTAAGCC-1,myCAF,0
XT11_CTTACCGGTATTCTCT,myCAF,0
XH_ZY-1174-06_CCCATACGTTTCGCTC-1,myCAF,0


In [25]:
f.obs.to_csv('../data/deconvolution/bayesprism/inputs/sc_ref_500_per_annotations.csv', sep=',')

In [19]:
df = pd.DataFrame(data=f.X.toarray(), columns=f.var.index.to_list(),
                  index=f.obs.index.to_list(), dtype=int)
df = df.transpose()
df.index.name = 'Gene'

df

,XT9_CACATTTAGAGTAATC,XT15_CTACCCAGTCACTGGC,XT2_ATTCTACAGTAATCCC,XT2_CTCGGGATCATAACCG,XT2_GGTGCGTTCAAACCAC,XG9903_filtered_gene_bc_matrices_h5.h5_GTACGTACAGACAAGC-1,XT2_CGCTTCATCCCTTGCA,XT12_CTAACTTCAGCCTGTG,X97727_filtered_gene_bc_matrices_h5.h5_CATCCACCAGCTGTAT-1,XT5_CTAGTGATCGTACGGC,...,XT11_GTAACTGGTTAAGATG,XT11_ACACCGGCAAGCCGCT,XT23_CGATGGCTCTGTACGA,XT11_AAGGTTCCACAACGTT,XT23_CAGAATCAGTGTTGAA,X87784_filtered_gene_bc_matrices_h5.h5_GGTGTTAGTTTAAGCC-1,XT11_CTTACCGGTATTCTCT,XH_ZY-1174-06_CCCATACGTTTCGCTC-1,XT9_GTTTCTATCAGGCAAG,XG9903_filtered_gene_bc_matrices_h5.h5_CTGAAGTTCCGTAGTA-1
Gene,,,,,,,,,,,,,,,,,,,,,
A1BG,1,0,0,0,0,0,0,0,0,0,...,1,0,1,1,1,1,0,0,0,2
A1CF,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A2M,0,1,0,0,0,0,0,1,0,0,...,1,6,0,11,1,0,2,0,0,1
A2ML1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
A4GALT,1,0,0,0,0,0,0,0,0,0,...,0,3,1,0,0,1,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZWINT,0,0,0,0,0,0,0,1,0,0,...,0,0,0,0,0,0,0,0,0,0
ZXDC,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,1,0,0,0,0,0
ZYG11B,1,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,0,0,0,0


In [20]:
df.to_csv('../data/deconvolution/bayesprism/inputs/sc_ref_500_per_counts.txt', sep='\t')

In [21]:
bulk_fps = sorted(listfiles('../data/bulk_rna_seq/', regex=r'bulk_rna_seq/[^/]+_counts.txt$'))
bulk_fps

['../data/bulk_rna_seq/bailey_counts.txt',
 '../data/bulk_rna_seq/cptac_counts.txt',
 '../data/bulk_rna_seq/kirby_counts.txt',
 '../data/bulk_rna_seq/tcga_counts.txt']

In [24]:
for fp in bulk_fps:
    dataset = fp.split('/')[-1].split('_')[0]
    df = pd.read_csv(fp, sep='\t', index_col=0)
    df['gene'] = df.index.to_list()
    df = df.groupby('gene').mean().astype(int)
    df.index.name = 'Gene'
    df = df.loc[sorted(genes), :]
    print(dataset, df.shape)
    df.to_csv(f'../data/deconvolution/bayesprism/inputs/{dataset}_counts.txt', sep='\t')

bailey (14048, 92)
cptac (14048, 140)
kirby (14048, 51)
tcga (14048, 177)
